# SSC Diatoms Extraction for Nudging Atlantis

This notebook explores using `dask.distributed` to extract diatom and lon/lat fields
from day-averaged SalishSeaCast hindcast files to create long time series files that
can be processed to generate "nudging" fields for Atlantis.

The packages required by this notebook are defined in the `conda` environment file `analysis-doug/notebooks/dask-expts/environment.yaml`
along with instructions on how to create an isolated environment containing the latest versions of those packages.
The `analysis-doug/notebooks/dask-expts/requirements.txt` contains a complete list of all the packages
(top level and dependencies) and their versions that were used most recently for work in this notebook.

To run this Jupyter Lab headless on port 8843 on `salish`,
launch `jupyter lab` there with with:

    jupyter lab --no-browser --ip $(hostname -f) --port=8843

and set up an `ssh` tunnel to connect to the kernel on port 8843 on `salish` from my local machine's port 4343 with:

    ssh -N -L 4343:salish:8843 salish

and finally launch the lab UI in my local browser with:

    http://localhost:4343/

## Setup

Python imports:

In [3]:
from pathlib import Path
import sys

import arrow
import dask.distributed
import netCDF4
import numpy
import xarray

Python and library versions:

In [4]:
print(f"Python {sys.version=}")
print(f"{xarray.__version__=}")
print(f"{dask.distributed.__version__=}")
print(f"{netCDF4.__version__=}")
print(f"{arrow.__version__=}")

Python sys.version='3.10.2 | packaged by conda-forge | (main, Jan 14 2022, 08:02:09) [GCC 9.4.0]'
xarray.__version__='0.20.2'
dask.distributed.__version__='2021.12.0'
netCDF4.__version__='1.5.8'
arrow.__version__='1.2.1'


## Source Datasets

The datasets that we want to extract the diatoms field from are stored in
`/results2/SalishSea/nowcast-green.201905/`.
Each day's run result are stored in a sub-directory with the name pattern `ddmmmyy/`;
e.g. `01jan07/`.
The model output file containing the day-averaged diatoms field has the name pattern
`SalishSea_1d_yyyymmdd_yyyymmdd_ptrc_T.nc`;
e.g. `SalishSea_1d_20070101_20070101_ptrc_T.nc`.

Let's take a look at one of those datasets:

In [5]:
results_archive = Path("/results2/SalishSea/nowcast-green.201905/")
start_date = arrow.get("2007-01-01")
ddmmmyy = start_date.format("DDMMMYY").lower()
yyyymmdd = start_date.format("YYYYMMDD")

ds_path = results_archive/ddmmmyy/f"SalishSea_1d_{yyyymmdd}_{yyyymmdd}_ptrc_T.nc"
ds = xarray.open_dataset(ds_path)

ds

<xarray.Dataset>
Dimensions:                       (time_counter: 1, y: 898, x: 398, deptht: 40)
Coordinates:
  * time_counter                  (time_counter) datetime64[ns] 2007-01-01T12...
    nav_lat                       (y, x) float32 ...
    nav_lon                       (y, x) float32 ...
  * deptht                        (deptht) float32 0.5 1.5 2.5 ... 414.5 441.5
Dimensions without coordinates: y, x
Data variables:
    nitrate                       (time_counter, deptht, y, x) float32 ...
    ammonium                      (time_counter, deptht, y, x) float32 ...
    silicon                       (time_counter, deptht, y, x) float32 ...
    diatoms                       (time_counter, deptht, y, x) float32 ...
    flagellates                   (time_counter, deptht, y, x) float32 ...
    ciliates                      (time_counter, deptht, y, x) float32 ...
    microzooplankton              (time_counter, deptht, y, x) float32 ...
    dissolved_organic_nitrogen    (time_counter, deptht, y, x) float32 ...
    particulate_organic_nitrogen  (time_counter, deptht, y, x) float32 ...
    biogenic_silicon              (time_counter, deptht, y, x) float32 ...
    mesozooplankton               (time_counter, deptht, y, x) float32 ...
Attributes:
    name:         SalishSea_1h_20070101_20070331_ptrc_T
    description:  biogeochemical variables
    title:        biogeochemical variables
    Conventions:  CF-1.6
    timeStamp:    2019-Sep-28 13:29:23 GMT
    uuid:         ce699af5-6953-4019-93ee-2208e1b3836b

Drop the variables that we don't need:


In [6]:
keep_vars = {"diatoms"}
drop_vars = {v for v in ds.data_vars} - keep_vars
ds = xarray.open_dataset(ds_path, drop_variables=drop_vars)

ds

<xarray.Dataset>
Dimensions:       (time_counter: 1, y: 898, x: 398, deptht: 40)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2007-01-01T12:00:00
    nav_lat       (y, x) float32 ...
    nav_lon       (y, x) float32 ...
  * deptht        (deptht) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
Dimensions without coordinates: y, x
Data variables:
    diatoms       (time_counter, deptht, y, x) float32 ...
Attributes:
    name:         SalishSea_1h_20070101_20070331_ptrc_T
    description:  biogeochemical variables
    title:        biogeochemical variables
    Conventions:  CF-1.6
    timeStamp:    2019-Sep-28 13:29:23 GMT
    uuid:         ce699af5-6953-4019-93ee-2208e1b3836b

## Build Extraction Dataset

Build a new dataset with the variables we want from the source dataset(s)
using the coordinates, variables, and dataset attributes that we want.

### Coordinates

In [7]:
time = xarray.DataArray(
    name="time",
    data=ds.time_counter.data,
    coords={
        "time": ds.time_counter.data,
    },
    attrs={
        "standard_name": "time",
        "long_name": "Time Axis",
        "comment": (
            "time values are UTC at the centre of the intervals over which the calculated model results are averaged;"
            "e.g. the field average values for 01 January 2007 have a time value of 2007-01-01 12:00:00Z"
        ),
        # time_origin and units are provided by encoding when dataset is written to netCDF file
    }
)

time

<xarray.DataArray 'time' (time: 1)>
array(['2007-01-01T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01T12:00:00
Attributes:
    standard_name:  time
    long_name:      Time Axis
    comment:        time values are UTC at the centre of the intervals over w...

In [8]:
depths = xarray.DataArray(
    name="depth",
    data=ds.deptht.data,
    coords={
        "depth": ds.deptht.data,
    },
    attrs={
        "standard_name": "sea_floor_depth",
        "long_name": "Sea Floor Depth",
        "units": "metres"
    }
)

depths

<xarray.DataArray 'depth' (depth: 40)>
array([  0.5000003,   1.5000031,   2.5000114,   3.5000305,   4.5000706,
         5.5001507,   6.5003104,   7.500623 ,   8.501236 ,   9.502433 ,
        10.5047655,  11.509312 ,  12.518167 ,  13.535412 ,  14.568982 ,
        15.634288 ,  16.761173 ,  18.007135 ,  19.481785 ,  21.389978 ,
        24.100256 ,  28.229916 ,  34.685757 ,  44.517723 ,  58.484333 ,
        76.58559  ,  98.06296  , 121.866516 , 147.08946  , 173.11449  ,
       199.57304  , 226.2603   , 253.06664  , 279.93454  , 306.8342   ,
       333.75018  , 360.67453  , 387.6032   , 414.5341   , 441.4661   ],
      dtype=float32)
Coordinates:
  * depth    (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
Attributes:
    standard_name:  sea_floor_depth
    long_name:      Sea Floor Depth
    units:          metres

In [9]:
grid_y = xarray.DataArray(
    name="gridY",
    data=ds.y.data,
    coords={
        "gridY": ds.y.data,
    },
    attrs={
        "standard_name": "y",
        "long_name": "Grid Y",
        "units": "count",
        "comment": "gridY values are grid indices in the model y-direction",
    }
)

grid_y

<xarray.DataArray 'gridY' (gridY: 898)>
array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
...
       650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662,
       663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675,
       676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688,
       689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701,
       702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714,
       715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727,
       728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740,
       741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753,
       754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766,
       767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779,
       780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792,
       793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805,
       806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818,
       819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831,
       832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844,
       845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857,
       858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870,
       871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883,
       884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896,
       897])
Coordinates:
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
Attributes:
    standard_name:  y
    long_name:      Grid Y
    units:          count
    comment:        gridY values are grid indices in the model y-direction

In [10]:
grid_x = xarray.DataArray(
    name="gridX",
    data=ds.x.data,
    coords={
        "gridX": ds.x.data,
    },
    attrs={
        "standard_name": "x",
        "long_name": "Grid X",
        "units": "count",
        "comment": "gridX values are grid indices in the model x-direction",
    }
)

grid_x

<xarray.DataArray 'gridX' (gridX: 398)>
array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
       260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272,
       273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285,
       286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298,
       299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311,
       312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324,
       325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337,
       338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350,
       351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
       364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376,
       377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389,
       390, 391, 392, 393, 394, 395, 396, 397])
Coordinates:
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  x
    long_name:      Grid X
    units:          count
    comment:        gridX values are grid indices in the model x-direction

### Variables

In [11]:
diatoms = xarray.DataArray(
    name="diatoms",
    data=ds.diatoms.data,
    coords={
        "time": time,
        "depth": depths,
        "gridY": grid_y,
        "gridX": grid_x,
    },
    attrs={
        "standard_name": "mole_concentration_of_diatoms_expressed_as_nitrogen_in_sea_water",
        "long_name": "Diatoms Concentration",
        "units": "mmol m-3"
    }
)

diatoms

<xarray.DataArray 'diatoms' (time: 1, depth: 40, gridY: 898, gridX: 398)>
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01T12:00:00
  * depth    (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  mole_concentration_of_diatoms_expressed_as_nitrogen_in_se...
    long_name:      Diatoms Concentration
    units:          mmol m-3

The longitudes and latitudes fields (`nav_lon` and `nav_lat`) in the SalishSeaCast results datasets
are a bit weird due the land processor elimination optimization that we use in NEMO to avoid
spending computational effort on grid cells that contain only land.
That optimization results in the longitude and latitude values being set to -1 in those cells.
So, instead we'll grab the lon/lat fields from the model grid bathymetry dataset
on ERDDAP [https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02.html](https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02.html):

In [12]:
grid_url = "https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02"
grid_ds = xarray.open_dataset(grid_url)

grid_ds

<xarray.Dataset>
Dimensions:     (gridY: 898, gridX: 398)
Coordinates:
  * gridY       (gridY) int32 0 1 2 3 4 5 6 7 ... 891 892 893 894 895 896 897
  * gridX       (gridX) int16 0 1 2 3 4 5 6 7 ... 391 392 393 394 395 396 397
Data variables:
    bathymetry  (gridY, gridX) float64 ...
    latitude    (gridY, gridX) float64 ...
    longitude   (gridY, gridX) float64 ...
Attributes: (12/21)
    acknowledgement:           MEOPAR, ONC, Compute Canada, CHS, NOAA, USGS\n...
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              Salish Sea MEOPAR Project Contributors
    creator_url:               https://salishsea-meopar-docs.readthedocs.io/
    ...                        ...
    references:                https://bitbucket.org/salishsea/nemo-forcing/r...
    source:                    https://bitbucket.org/salishsea/tools/src/tip/...
    sourceUrl:                 (local files)
    standard_name_vocabulary:  CF Standard Name Table v29
    summary:                   Salish Sea NEMO Model Grid, Geo-location and B...
    title:                     Salish Sea NEMO Model Grid, Geo-location and B...

In [13]:
lons = xarray.DataArray(
    name="longitude",
    data=grid_ds.longitude.data,
    coords={
        "gridY": grid_y,
        "gridX": grid_x,
    },
    attrs={
        "standard_name": "longitude",
        "long_name": "Longitude",
        "units": "degrees_east"
    }
)

lons

<xarray.DataArray 'longitude' (gridY: 898, gridX: 398)>
array([[-123.42943573, -123.42411804, -123.41880035, ..., -121.32898712,
        -121.32366943, -121.31835175],
       [-123.43196869, -123.42677307, -123.42152405, ..., -121.33235931,
        -121.32704163, -121.32172394],
       [-123.43463898, -123.42948151, -123.42427063, ..., -121.33573151,
        -121.33041382, -121.32509613],
       ...,
       [-126.39330292, -126.38764191, -126.3819809 , ..., -124.34471893,
        -124.340065  , -124.3354187 ],
       [-126.39678955, -126.39109802, -126.38541412, ..., -124.34797668,
        -124.34333801, -124.3387146 ],
       [-126.40029144, -126.39456177, -126.38883209, ..., -124.35121918,
        -124.34658813, -124.34198761]])
Coordinates:
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east

In [14]:
lats = xarray.DataArray(
    name="latitude",
    data=grid_ds.latitude.data,
    coords={
        "gridY": grid_y,
        "gridX": grid_x,
    },
    attrs={
        "standard_name": "latitude",
        "long_name": "Latitude",
        "units": "degrees_north"
    }
)

lats

<xarray.DataArray 'latitude' (gridY: 898, gridX: 398)>
array([[46.85966492, 46.86154556, 46.86342621, ..., 47.59721375,
        47.59906769, 47.60092163],
       [46.86278915, 46.86481476, 46.86677933, ..., 47.60125732,
        47.60311127, 47.60496521],
       [46.86606979, 46.86814499, 46.87015915, ..., 47.60529709,
        47.60715485, 47.60900879],
       ...,
       [50.38191605, 50.38397598, 50.38602448, ..., 51.09400177,
        51.09560776, 51.09720612],
       [50.38591766, 50.38798523, 50.39004135, ..., 51.09781265,
        51.0994072 , 51.10100174],
       [50.38992691, 50.39200592, 50.39406967, ..., 51.10162354,
        51.10321808, 51.10480118]])
Coordinates:
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north

In [15]:
extracted_ds = xarray.Dataset(
    coords={
        "time": time,
        "depth": depths,
        "gridY": grid_y,
        "gridX": grid_x,
    },
    data_vars={
        "longitude": lons,
        "latitude": lats,
        "diatoms": diatoms,
    },
    attrs={
        "name": "SalishSeaCast_day_avg_diatoms_20070101_20070101",
        "description": "Day-averaged diatoms biomass extracted from SalishSeaCast v201905 hindcast",
        "history": f"{arrow.now('PST').format('YYYY-MM-DD HH:mm')}: Generated by analysis-doug/notebooks/dask-expts/atlantis_nudge_diatoms.ipynb",
    },
)

extracted_ds

<xarray.Dataset>
Dimensions:    (gridY: 898, gridX: 398, time: 1, depth: 40)
Coordinates:
  * gridY      (gridY) int64 0 1 2 3 4 5 6 7 ... 890 891 892 893 894 895 896 897
  * gridX      (gridX) int64 0 1 2 3 4 5 6 7 ... 390 391 392 393 394 395 396 397
  * time       (time) datetime64[ns] 2007-01-01T12:00:00
  * depth      (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
Data variables:
    longitude  (gridY, gridX) float64 -123.4 -123.4 -123.4 ... -124.3 -124.3
    latitude   (gridY, gridX) float64 46.86 46.86 46.86 46.87 ... 51.1 51.1 51.1
    diatoms    (time, depth, gridY, gridX) float32 nan nan nan ... nan nan nan
Attributes:
    name:         SalishSeaCast_day_avg_diatoms_20070101_20070101
    description:  Day-averaged diatoms biomass extracted from SalishSeaCast v...
    history:      2022-01-20 12:06: Generated by analysis-doug/notebooks/dask...

## Store the Extracted Dataset as a netCDF File

The main thing here is that we need to setup a nested `dict` of encoding information
to tell `xarray` and the underlying `netCDF4` library some details about how to store
the dataset as a netCDF4 file.

In [16]:
nc_path = Path("/tmp/SalishSeaCast_day_avg_diatoms_20070101_20070101.nc")

encoding = {
    "time": {
        "dtype": numpy.single,
        "units": "days since 2007-01-01 12:00:00",
        "chunksizes": [1],
    },
    "depth": {"dtype": numpy.single, "chunksizes": [depths.size]},
    "gridY": {"dtype": int, "chunksizes": [grid_y.size]},
    "gridX": {"dtype": int, "chunksizes": [grid_x.size]},
    "diatoms": {"dtype": numpy.single, "chunksizes": [1] + [c_array.size for c_name, c_array in diatoms.coords.items() if c_name != "time"]},
    "longitude": {"dtype": numpy.single, "chunksizes": [c_array.size for c_array in lons.coords.values()]},
    "latitude": {"dtype": numpy.single, "chunksizes": [c_array.size for c_array in lats.coords.values()]},
}

In [17]:
extracted_ds.to_netcdf(nc_path, format="NETCDF4_CLASSIC", encoding=encoding, unlimited_dims="time")

In [18]:
ls -lh /tmp/SalishSeaCast_day_avg_diatoms_20070101_20070101.nc

-rw-rw-r-- 1 doug doug 58M Jan 20 12:06 /tmp/SalishSeaCast_day_avg_diatoms_20070101_20070101.nc


In [19]:
!ncdump -cs /tmp/SalishSeaCast_day_avg_diatoms_20070101_20070101.nc

netcdf SalishSeaCast_day_avg_diatoms_20070101_20070101 {
dimensions:
	time = UNLIMITED ; // (1 currently)
	gridY = 898 ;
	gridX = 398 ;
	depth = 40 ;
variables:
	int gridY(gridY) ;
		gridY:standard_name = "y" ;
		gridY:long_name = "Grid Y" ;
		gridY:units = "count" ;
		gridY:comment = "gridY values are grid indices in the model y-direction" ;
		gridY:_Storage = "chunked" ;
		gridY:_ChunkSizes = 898 ;
		gridY:_Endianness = "little" ;
	int gridX(gridX) ;
		gridX:standard_name = "x" ;
		gridX:long_name = "Grid X" ;
		gridX:units = "count" ;
		gridX:comment = "gridX values are grid indices in the model x-direction" ;
		gridX:_Storage = "chunked" ;
		gridX:_ChunkSizes = 398 ;
		gridX:_Endianness = "little" ;
	float longitude(gridY, gridX) ;
		longitude:_FillValue = NaNf ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude" ;
		longitude:units = "degrees_east" ;
		longitude:_Storage = "chunked" ;
		longitude:_ChunkSizes = 898, 398 ;
		longitude:_Endianness = "little

In [20]:
ds.close()
grid_ds.close()

## Scaling and Refactoring

Now we want to scale the processing to pull diatoms fields from an arbitrary range of dates
of SalishSeaCast hindcast files.
We'll use [dask]() and [dask.distributed]() to spread the work of processing many day's dataset
over multiple processors/cores.

We also want to abstract some repetitious code above into functions.

### Source Dataset Paths

In [111]:
results_archive = Path("/results2/SalishSea/nowcast-green.201905/")
start_date = arrow.get("2007-01-01")
end_date = arrow.get("2007-01-31")

In [69]:
def ddmmmyy(arrow_date):
    """Return an Arrow date as a string formatted as lower-cased `ddmmmyy`."""
    return arrow_date.format("DDMMMYY").lower()

In [70]:
def yyyymmdd(arrow_date):
    """Return an Arrow date as a string of digits formatted as `yyyymmdd`."""
    return arrow_date.format("YYYYMMDD")

In [112]:
date_range = arrow.Arrow.range("days", start_date, end_date)
ds_paths = [results_archive/ddmmmyy(day)/f"SalishSea_1d_{yyyymmdd(day)}_{yyyymmdd(day)}_ptrc_T.nc" for day in date_range]

ds_paths

[PosixPath('/results2/SalishSea/nowcast-green.201905/01jan07/SalishSea_1d_20070101_20070101_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/02jan07/SalishSea_1d_20070102_20070102_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/03jan07/SalishSea_1d_20070103_20070103_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/04jan07/SalishSea_1d_20070104_20070104_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/05jan07/SalishSea_1d_20070105_20070105_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/06jan07/SalishSea_1d_20070106_20070106_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/07jan07/SalishSea_1d_20070107_20070107_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/08jan07/SalishSea_1d_20070108_20070108_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/09jan07/SalishSea_1d_20070109_20070109_ptrc_T.nc'),
 PosixPath('/results2/SalishSea/nowcast-green.201905/10

### Start `dask` Cluster

In [63]:
client = dask.distributed.Client(
    n_workers=4, threads_per_worker=10, processes=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 40,Total memory: 62.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38617,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 40
Started: Just now,Total memory: 62.66 GiB
Comm: tcp://127.0.0.1:46007,Total threads: 10
Dashboard: http://127.0.0.1:37073/status,Memory: 15.66 GiB
Nanny: tcp://127.0.0.1:36733,


### Load Dataset Collection Metadata

In [72]:
chunks = {"time_counter": 1, "deptht": 40, "y": 898, "x": 398}

In [73]:
def open_reduced_dataset(ds_paths, chunks, keep_vars):
    # Use 1st dataset path to calculate the set of variables to drop
    with xarray.open_dataset(ds_paths[0], chunks=chunks) as ds:
        drop_vars = {var for var in ds.data_vars} - keep_vars
    # Return dataset with only variables of interest for full list of paths
    # This triggers a dask task graph computation
    return xarray.open_mfdataset(ds_paths, chunks=chunks, drop_variables=drop_vars)

In [113]:
%%time
ds = open_reduced_dataset(ds_paths, chunks, {"diatoms"})

ds

CPU times: user 1.38 s, sys: 173 ms, total: 1.55 s
Wall time: 3.59 s


<xarray.Dataset>
Dimensions:       (time_counter: 31, y: 898, x: 398, deptht: 40)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2007-01-01T12:00:00 ... 2007-...
    nav_lat       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
  * deptht        (deptht) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
Dimensions without coordinates: y, x
Data variables:
    diatoms       (time_counter, deptht, y, x) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
Attributes:
    name:         SalishSea_1h_20070101_20070331_ptrc_T
    description:  biogeochemical variables
    title:        biogeochemical variables
    Conventions:  CF-1.6
    timeStamp:    2019-Sep-28 13:29:23 GMT
    uuid:         ce699af5-6953-4019-93ee-2208e1b3836b

### Coordinates

In [75]:
def calc_coord_array(name, source_data_var, attrs):
    return xarray.DataArray(
        name=name,
        data=source_data_var.data,
        coords={name: source_data_var.data},
        attrs=attrs
    )

In [114]:
time = calc_coord_array(
    "time", ds.time_counter,
    attrs={
        "standard_name": "time",
        "long_name": "Time Axis",
        "comment": (
            "time values are UTC at the centre of the intervals over which the calculated model results are averaged;"
            "e.g. the field average values for 01 January 2007 have a time value of 2007-01-01 12:00:00Z"
        ),
        # time_origin and units are provided by encoding when dataset is written to netCDF file
    }
)

time

<xarray.DataArray 'time' (time: 31)>
array(['2007-01-01T12:00:00.000000000', '2007-01-02T12:00:00.000000000',
       '2007-01-03T12:00:00.000000000', '2007-01-04T12:00:00.000000000',
       '2007-01-05T12:00:00.000000000', '2007-01-06T12:00:00.000000000',
       '2007-01-07T12:00:00.000000000', '2007-01-08T12:00:00.000000000',
       '2007-01-09T12:00:00.000000000', '2007-01-10T12:00:00.000000000',
       '2007-01-11T12:00:00.000000000', '2007-01-12T12:00:00.000000000',
       '2007-01-13T12:00:00.000000000', '2007-01-14T12:00:00.000000000',
       '2007-01-15T12:00:00.000000000', '2007-01-16T12:00:00.000000000',
       '2007-01-17T12:00:00.000000000', '2007-01-18T12:00:00.000000000',
       '2007-01-19T12:00:00.000000000', '2007-01-20T12:00:00.000000000',
       '2007-01-21T12:00:00.000000000', '2007-01-22T12:00:00.000000000',
       '2007-01-23T12:00:00.000000000', '2007-01-24T12:00:00.000000000',
       '2007-01-25T12:00:00.000000000', '2007-01-26T12:00:00.000000000',
       '2007-01-27T12:00:00.000000000', '2007-01-28T12:00:00.000000000',
       '2007-01-29T12:00:00.000000000', '2007-01-30T12:00:00.000000000',
       '2007-01-31T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01T12:00:00 ... 2007-01-31T12:00:00
Attributes:
    standard_name:  time
    long_name:      Time Axis
    comment:        time values are UTC at the centre of the intervals over w...

In [77]:
depths = calc_coord_array(
    "depth", ds.deptht,
    attrs={
        "standard_name": "sea_floor_depth",
        "long_name": "Sea Floor Depth",
        "units": "metres"
    }
)

depths

<xarray.DataArray 'depth' (depth: 40)>
array([  0.5000003,   1.5000031,   2.5000114,   3.5000305,   4.5000706,
         5.5001507,   6.5003104,   7.500623 ,   8.501236 ,   9.502433 ,
        10.5047655,  11.509312 ,  12.518167 ,  13.535412 ,  14.568982 ,
        15.634288 ,  16.761173 ,  18.007135 ,  19.481785 ,  21.389978 ,
        24.100256 ,  28.229916 ,  34.685757 ,  44.517723 ,  58.484333 ,
        76.58559  ,  98.06296  , 121.866516 , 147.08946  , 173.11449  ,
       199.57304  , 226.2603   , 253.06664  , 279.93454  , 306.8342   ,
       333.75018  , 360.67453  , 387.6032   , 414.5341   , 441.4661   ],
      dtype=float32)
Coordinates:
  * depth    (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
Attributes:
    standard_name:  sea_floor_depth
    long_name:      Sea Floor Depth
    units:          metres

In [78]:
grid_y = calc_coord_array(
    "gridY", ds.y,
    attrs={
        "standard_name": "y",
        "long_name": "Grid Y",
        "units": "count",
        "comment": "gridY values are grid indices in the model y-direction",
    }
)

grid_y

<xarray.DataArray 'gridY' (gridY: 898)>
array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
...
       650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662,
       663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675,
       676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688,
       689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701,
       702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714,
       715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727,
       728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740,
       741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753,
       754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766,
       767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779,
       780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792,
       793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805,
       806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818,
       819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831,
       832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844,
       845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857,
       858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870,
       871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883,
       884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896,
       897])
Coordinates:
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
Attributes:
    standard_name:  y
    long_name:      Grid Y
    units:          count
    comment:        gridY values are grid indices in the model y-direction

In [79]:
grid_x = calc_coord_array(
    "gridX", ds.x,
    attrs={
        "standard_name": "x",
        "long_name": "Grid X",
        "units": "count",
        "comment": "gridX values are grid indices in the model x-direction",
    }
)

grid_x

<xarray.DataArray 'gridX' (gridX: 398)>
array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
       260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272,
       273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285,
       286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298,
       299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311,
       312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324,
       325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337,
       338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350,
       351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
       364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376,
       377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389,
       390, 391, 392, 393, 394, 395, 396, 397])
Coordinates:
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  x
    long_name:      Grid X
    units:          count
    comment:        gridX values are grid indices in the model x-direction

### Variables

In [80]:
def calc_var_array(name, source_data_var, coords, attrs):
    return xarray.DataArray(
        name=name,
        data=source_data_var.data,
        coords=coords,
        attrs=attrs
    )

In [115]:
diatoms = calc_var_array(
    "diatoms", ds.diatoms,
    coords={
        "time": time,
        "depth": depths,
        "gridY": grid_y,
        "gridX": grid_x,
    },
    attrs={
        "standard_name": "mole_concentration_of_diatoms_expressed_as_nitrogen_in_sea_water",
        "long_name": "Diatoms Concentration",
        "units": "mmol m-3"
    }
)

diatoms

,Array,Chunk
Bytes,1.65 GiB,54.54 MiB
Shape,"(31, 40, 898, 398)","(1, 40, 898, 398)"
Count,93 Tasks,31 Chunks
Type,float32,numpy.ndarray


In [82]:
grid_url = "https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02"
grid_ds = xarray.open_dataset(grid_url)

grid_ds

<xarray.Dataset>
Dimensions:     (gridY: 898, gridX: 398)
Coordinates:
  * gridY       (gridY) int32 0 1 2 3 4 5 6 7 ... 891 892 893 894 895 896 897
  * gridX       (gridX) int16 0 1 2 3 4 5 6 7 ... 391 392 393 394 395 396 397
Data variables:
    bathymetry  (gridY, gridX) float64 ...
    latitude    (gridY, gridX) float64 ...
    longitude   (gridY, gridX) float64 ...
Attributes: (12/21)
    acknowledgement:           MEOPAR, ONC, Compute Canada, CHS, NOAA, USGS\n...
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              Salish Sea MEOPAR Project Contributors
    creator_url:               https://salishsea-meopar-docs.readthedocs.io/
    ...                        ...
    references:                https://bitbucket.org/salishsea/nemo-forcing/r...
    source:                    https://bitbucket.org/salishsea/tools/src/tip/...
    sourceUrl:                 (local files)
    standard_name_vocabulary:  CF Standard Name Table v29
    summary:                   Salish Sea NEMO Model Grid, Geo-location and B...
    title:                     Salish Sea NEMO Model Grid, Geo-location and B...

In [83]:
lons = calc_var_array(
    "longitude", grid_ds.longitude,
    coords={
        "gridY": grid_y,
        "gridX": grid_x,
    },
    attrs={
        "standard_name": "longitude",
        "long_name": "Longitude",
        "units": "degrees_east"
    }
)

lons

<xarray.DataArray 'longitude' (gridY: 898, gridX: 398)>
array([[-123.42943573, -123.42411804, -123.41880035, ..., -121.32898712,
        -121.32366943, -121.31835175],
       [-123.43196869, -123.42677307, -123.42152405, ..., -121.33235931,
        -121.32704163, -121.32172394],
       [-123.43463898, -123.42948151, -123.42427063, ..., -121.33573151,
        -121.33041382, -121.32509613],
       ...,
       [-126.39330292, -126.38764191, -126.3819809 , ..., -124.34471893,
        -124.340065  , -124.3354187 ],
       [-126.39678955, -126.39109802, -126.38541412, ..., -124.34797668,
        -124.34333801, -124.3387146 ],
       [-126.40029144, -126.39456177, -126.38883209, ..., -124.35121918,
        -124.34658813, -124.34198761]])
Coordinates:
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east

In [84]:
lats = calc_var_array(
    "latitude", grid_ds.latitude,
    coords={
        "gridY": grid_y,
        "gridX": grid_x,
    },
    attrs={
        "standard_name": "latitude",
        "long_name": "Latitude",
        "units": "degrees_north"
    }
)

lats

<xarray.DataArray 'latitude' (gridY: 898, gridX: 398)>
array([[46.85966492, 46.86154556, 46.86342621, ..., 47.59721375,
        47.59906769, 47.60092163],
       [46.86278915, 46.86481476, 46.86677933, ..., 47.60125732,
        47.60311127, 47.60496521],
       [46.86606979, 46.86814499, 46.87015915, ..., 47.60529709,
        47.60715485, 47.60900879],
       ...,
       [50.38191605, 50.38397598, 50.38602448, ..., 51.09400177,
        51.09560776, 51.09720612],
       [50.38591766, 50.38798523, 50.39004135, ..., 51.09781265,
        51.0994072 , 51.10100174],
       [50.38992691, 50.39200592, 50.39406967, ..., 51.10162354,
        51.10321808, 51.10480118]])
Coordinates:
  * gridY    (gridY) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * gridX    (gridX) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north

### Extraction Dataset

In [116]:
extracted_ds_name = f"SalishSeaCast_day_avg_diatoms_{yyyymmdd(start_date)}_{yyyymmdd(end_date)}"
extracted_ds = xarray.Dataset(
    coords={
        "time": time,
        "depth": depths,
        "gridY": grid_y,
        "gridX": grid_x,
    },
    data_vars={
        "longitude": lons,
        "latitude": lats,
        "diatoms": diatoms,
    },
    attrs={
        "name": extracted_ds_name,
        "description": "Day-averaged diatoms biomass extracted from SalishSeaCast v201905 hindcast",
        "history": f"{arrow.now('PST').format('YYYY-MM-DD HH:mm')}: Generated by analysis-doug/notebooks/dask-expts/atlantis_nudge_diatoms.ipynb",
    },
)

extracted_ds

,Array,Chunk
Bytes,1.65 GiB,54.54 MiB
Shape,"(31, 40, 898, 398)","(1, 40, 898, 398)"
Count,93 Tasks,31 Chunks
Type,float32,numpy.ndarray


### Encoding for netCDF Storage

In [86]:
def calc_coord_encoding(ds, coord):
    match coord:
        case "time":
            return {
                "dtype": numpy.single,
                "units": "days since 2007-01-01 12:00:00",
                "chunksizes": [1],
            }
        case "depth":
            return {
                "dtype": numpy.single,
                "chunksizes": [ds.coords[coord].size]
            }
        case _:
            return {
                "dtype": int,
                "chunksizes": [ds.coords[coord].size]
            }

In [87]:
def calc_var_encoding(var, coord_names):
    chunksizes = []
    for c_name in coord_names:
        try:
            chunksizes.append(var.coords[c_name].size)
        except KeyError:
            pass
    if "time" in var.coords:
        chunksizes[0] = 1
    return {
        "dtype": numpy.single,
        "chunksizes": chunksizes,
    }

In [88]:
encoding = {}
for coord in extracted_ds.coords:
    encoding[coord] = calc_coord_encoding(extracted_ds, coord)
for v_name, v_array in extracted_ds.data_vars.items():
    encoding[v_name] = calc_var_encoding(v_array, ("time", "depth", "gridY", "gridX"))

encoding

{'gridY': {'dtype': int, 'chunksizes': [898]},
 'gridX': {'dtype': int, 'chunksizes': [398]},
 'time': {'dtype': numpy.float32,
  'units': 'days since 2007-01-01 12:00:00',
  'chunksizes': [1]},
 'depth': {'dtype': numpy.float32, 'chunksizes': [40]},
 'longitude': {'dtype': numpy.float32, 'chunksizes': [898, 398]},
 'latitude': {'dtype': numpy.float32, 'chunksizes': [898, 398]},
 'diatoms': {'dtype': numpy.float32, 'chunksizes': [1, 40, 898, 398]}}

### Store Extraction as netCDF

In [117]:
%%time
nc_path = Path("/tmp/", f"{extracted_ds_name}.nc")
extracted_ds.to_netcdf(nc_path, format="NETCDF4_CLASSIC", encoding=encoding, unlimited_dims="time")

CPU times: user 433 ms, sys: 68.5 ms, total: 501 ms
Wall time: 3.71 s


In [118]:
ls -lh {nc_path}

-rw-rw-r-- 1 doug doug 1.7G Jan 20 16:08 /tmp/SalishSeaCast_day_avg_diatoms_20070101_20070131.nc


In [93]:
!ncdump -cs {nc_path}

netcdf SalishSeaCast_day_avg_diatoms_20070101_20070131 {
dimensions:
	time = UNLIMITED ; // (31 currently)
	gridY = 898 ;
	gridX = 398 ;
	depth = 40 ;
variables:
	int gridY(gridY) ;
		gridY:standard_name = "y" ;
		gridY:long_name = "Grid Y" ;
		gridY:units = "count" ;
		gridY:comment = "gridY values are grid indices in the model y-direction" ;
		gridY:_Storage = "chunked" ;
		gridY:_ChunkSizes = 898 ;
		gridY:_Endianness = "little" ;
	int gridX(gridX) ;
		gridX:standard_name = "x" ;
		gridX:long_name = "Grid X" ;
		gridX:units = "count" ;
		gridX:comment = "gridX values are grid indices in the model x-direction" ;
		gridX:_Storage = "chunked" ;
		gridX:_ChunkSizes = 398 ;
		gridX:_Endianness = "little" ;
	float longitude(gridY, gridX) ;
		longitude:_FillValue = NaNf ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude" ;
		longitude:units = "degrees_east" ;
		longitude:_Storage = "chunked" ;
		longitude:_ChunkSizes = 898, 398 ;
		longitude:_Endianness = "littl

## Move to Python Module

The next step is to put the code in this section into a Python module for further
testing and profiling without the overhead and other complications of working
with lots of datasets in the notebook context.
Please see `atlantis_nudge_diatoms.py`.

The move to a module is also an early step in the design of the planned
Reshapr package for generalized processing of this type.

## Close Datasets

In [119]:
ds.close()
grid_ds.close()

## Shut Down `dask` Cluster

In [120]:
client.close()